# Data Validation for Bondora P2P Lending Using Python and TensorFlow Data Validation
### David Lowe
### August 30, 2021

SUMMARY: The project aims to construct a data validation flow using TensorFlow Data Validation (TFDV) and document the end-to-end steps using a template. The Bondora P2P Lending dataset is a binary classification situation where we attempt to predict one of the two possible outcomes.

INTRODUCTION: The Kaggle dataset owner retrieved this dataset from Bondora, a leading European peer-to-peer lending platform. The data comprises demographic and financial information of the borrowers and loan transactions. For investors, "peer-to-peer lending" or "P2P" offers an attractive way to diversify portfolios and enhance long-term performance. To make effective decisions, investors want to minimize the risk of default of each lending decision and realize the return that compensates for the risk. The retrieved data is a combined dataset of defaulted and non-defaulted loans between February 2009 and July 2021. We will try to predict the default risk using the available information.

Additional Notes: I adapted this workflow from the TensorFlow Data Validation tutorial on TensorFlow.org (https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic). The plan is to build a robust TFDV script for validating datasets in building machine learning models.

CONCLUSION: In this iteration, the data validation workflow helped to validate the features and structures of the training, validation, and test datasets. The workflow also generated statistics over different slices of data which can help track model and anomaly metrics.

Dataset Used: Kaggle Bondora P2P Lending Loan Data

Dataset ML Model: Binary classification with numerical and categorical attributes

Dataset Reference: https://www.kaggle.com/sid321axn/bondora-peer-to-peer-lending-loan-data

Dataset Attribute Description: https://www.bondora.com/en/public-reports

Data validation for a machine learning project generally can be broken down into the following tasks:

1. Prepare Environment
2. Generate and Visualize Training Data Statistics
3. Check Anomalies in Validation Dataset
4. Check Anomalies in Test Dataset
5. Check for Data Drift and Skew
6. Display Stats for Data Slices
7. Finalize the Schema

## Task 1 - Prepare Environment

### 1.a) Load libraries and modules

In [1]:
# Set the random seed number for reproducible results
RNG_SEED = 8

In [2]:
# Import packages
import os
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tempfile, urllib, zipfile
import tensorflow_data_validation as tfdv
from tensorflow.python.lib.io import file_io
from tensorflow_data_validation.utils import slicing_util
from tensorflow_metadata.proto.v0.statistics_pb2 import DatasetFeatureStatisticsList, DatasetFeatureStatistics
from tensorflow_metadata.proto.v0 import schema_pb2

### 1.b) Set up the controlling parameters and functions

In [3]:
# Begin the timer for the script processing
start_time_script = datetime.now()

# Set the percentage sizes for splitting the dataset
VAL_SET_RATIO = 0.2
TEST_SET_RATIO = 0.5

# Set TF's logger to only display errors to avoid internal warnings being shown
tf.get_logger().setLevel('ERROR')
print('TFDV version: {}'.format(tfdv.version.__version__))

TFDV version: 1.0.0


### 1.c) Load dataset

In [4]:
# Read CSV data into a dataframe and mark the missing data as NaN
dataset_path = 'https://dainesanalytics.com/datasets/kaggle-bondora-p2p-lending/LoanData_Bondora.csv'
df_dataset_import = pd.read_csv(dataset_path, header=0)

# Take a peek at the dataframe after import
print(df_dataset_import.head())

/home/tensorflowml/appdev/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (34,38,80,82,83,89,90) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


  ReportAsOfEOD                                LoanId  LoanNumber  \
0    2021-07-20  66AE108B-532B-4BB3-BAB7-0019A46412C1      483449   
1    2021-07-20  D152382E-A50D-46ED-8FF2-0053E0C86A70      378148   
2    2021-07-20  87342E13-66CB-483F-833A-007953E50C78      451831   
3    2021-07-20  87227056-6BF9-410C-98D1-008F788E122A      349381   
4    2021-07-20  2DDE6336-E466-4624-A337-00A0ED1A1468      443082   

           ListedOnUTC     BiddingStartedOn  BidsPortfolioManager  BidsApi  \
0  2016-03-23 16:07:19  2016-03-23 16:07:19                   970     1150   
1  2015-06-25 11:02:28  2015-06-25 11:02:28                  1295        0   
2  2016-01-14 10:00:21  2016-01-14 10:00:21                  2700      565   
3  2015-03-24 15:55:44  2015-03-24 15:55:44                  1115        0   
4  2015-12-17 10:12:00  2015-12-17 10:12:00                   305        0   

   BidsManual   UserName  NewCreditCustomer  ...  \
0         5.0   BO965519              False  ...   
1      1705.

In [5]:
# Standardize the class column to the name of target
df_dataset_import = df_dataset_import.rename(columns={'Status':'target'})

In [6]:
# Quickly handle some special data cleaning situations
df_dataset_import['NrOfDependants'] = df_dataset_import['NrOfDependants'].replace(['10Plus'], '11')
df_dataset_import['NrOfDependants'].fillna(0, inplace=True)
df_dataset_import['NrOfDependants'] = df_dataset_import['NrOfDependants'].astype('int')

In [7]:
df_dataset_import.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179235 entries, 0 to 179234
Data columns (total 112 columns):
 #    Column                                  Dtype  
---   ------                                  -----  
 0    ReportAsOfEOD                           object 
 1    LoanId                                  object 
 2    LoanNumber                              int64  
 3    ListedOnUTC                             object 
 4    BiddingStartedOn                        object 
 5    BidsPortfolioManager                    int64  
 6    BidsApi                                 int64  
 7    BidsManual                              float64
 8    UserName                                object 
 9    NewCreditCustomer                       bool   
 10   LoanApplicationStartedDate              object 
 11   LoanDate                                object 
 12   ContractEndDate                         object 
 13   FirstPaymentDate                        object 
 14   MaturityDate_Origi

### 1.d) Splitting Data into Sets

In [8]:
# Split the data further into training, validation, and test datasets
df_train, df_val_test = train_test_split(df_dataset_import, test_size=VAL_SET_RATIO, random_state=RNG_SEED)
df_val, df_test = train_test_split(df_val_test, test_size=TEST_SET_RATIO, random_state=RNG_SEED)

# Test data emulates the data that would be submitted for predictions, so it should not have the label column.
df_test = df_test.drop(['target'], axis=1)

print("Training dataset has {} records and {} columns.".format(df_train.shape[0], df_train.shape[1]))
print("Validation dataset has {} records and {} columns.".format(df_val.shape[0], df_val.shape[1]))
print("Test dataset has {} records and {} columns".format(df_test.shape[0], df_test.shape[1]))

Training dataset has 143388 records and 112 columns.
Validation dataset has 17923 records and 112 columns.
Test dataset has 17924 records and 111 columns


## Task 2 - Generate and Visualize Training Data Statistics

### 2.a) Remove Non-Essential Features

In [9]:
high_pct_null_features = []
total_records = df_train.shape[0]
for feature in df_train.columns:
    missing_records = df_train[feature].isnull().sum()
    missing_percentage = missing_records / total_records
    print('Feature', feature, 'has', missing_records,'missing rows. Percentage:', missing_percentage * 100)
    if missing_percentage >= 0.75:
        high_pct_null_features.append(feature)
print('Proposed lis of features for removal:', high_pct_null_features)

Feature ReportAsOfEOD has 0 missing rows. Percentage: 0.0
Feature LoanId has 0 missing rows. Percentage: 0.0
Feature LoanNumber has 0 missing rows. Percentage: 0.0
Feature ListedOnUTC has 0 missing rows. Percentage: 0.0
Feature BiddingStartedOn has 0 missing rows. Percentage: 0.0
Feature BidsPortfolioManager has 0 missing rows. Percentage: 0.0
Feature BidsApi has 0 missing rows. Percentage: 0.0
Feature BidsManual has 0 missing rows. Percentage: 0.0
Feature UserName has 0 missing rows. Percentage: 0.0
Feature NewCreditCustomer has 0 missing rows. Percentage: 0.0
Feature LoanApplicationStartedDate has 0 missing rows. Percentage: 0.0
Feature LoanDate has 0 missing rows. Percentage: 0.0
Feature ContractEndDate has 61098 missing rows. Percentage: 42.61026027282618
Feature FirstPaymentDate has 0 missing rows. Percentage: 0.0
Feature MaturityDate_Original has 0 missing rows. Percentage: 0.0
Feature MaturityDate_Last has 0 missing rows. Percentage: 0.0
Feature ApplicationSignedHour has 0 missi

In [10]:
# Define non-essential features to remove
features_to_remove = {'ReportAsOfEOD','LoanId','LoanNumber','ListedOnUTC','BiddingStartedOn',
                      'BidsPortfolioManager','BidsApi','BidsManual','UserName','LoanApplicationStartedDate',
                      'LoanDate','ContractEndDate','FirstPaymentDate','MaturityDate_Original','MaturityDate_Last',
                      'DateOfBirth','DateOfBirth','County','City','EmploymentPosition',
                      'WorkExperience','EL_V0','Rating_V0','EL_V1','Rating_V1','Rating_V2',
                      'CreditScoreEsEquifaxRisk','CreditScoreFiAsiakasTietoRiskGrade',
                      'GracePeriodStart','GracePeriodEnd','NextPaymentDate','DefaultDate'}

# Collect features to whitelist while computing the statistics
features_to_keep = [col for col in df_dataset_import.columns if (col not in features_to_remove)]

# Instantiate a StatsOptions class and define the feature_whitelist property
options_train = tfdv.StatsOptions(feature_allowlist=features_to_keep)

# Review the features to generate the statistics
print(options_train.feature_allowlist)
print('Number of Features:', len(options_train.feature_allowlist))

['NewCreditCustomer', 'ApplicationSignedHour', 'ApplicationSignedWeekday', 'VerificationType', 'LanguageCode', 'Age', 'Gender', 'Country', 'AppliedAmount', 'Amount', 'Interest', 'LoanDuration', 'MonthlyPayment', 'UseOfLoan', 'Education', 'MaritalStatus', 'NrOfDependants', 'EmploymentStatus', 'EmploymentDurationCurrentEmployer', 'OccupationArea', 'HomeOwnershipType', 'IncomeFromPrincipalEmployer', 'IncomeFromPension', 'IncomeFromFamilyAllowance', 'IncomeFromSocialWelfare', 'IncomeFromLeavePay', 'IncomeFromChildSupport', 'IncomeOther', 'IncomeTotal', 'ExistingLiabilities', 'LiabilitiesTotal', 'RefinanceLiabilities', 'DebtToIncome', 'FreeCash', 'MonthlyPaymentDay', 'ActiveScheduleFirstPaymentReached', 'PlannedPrincipalTillDate', 'PlannedInterestTillDate', 'LastPaymentOn', 'CurrentDebtDaysPrimary', 'DebtOccuredOn', 'CurrentDebtDaysSecondary', 'DebtOccuredOnForSecondary', 'ExpectedLoss', 'LossGivenDefault', 'ExpectedReturn', 'ProbabilityOfDefault', 'PrincipalOverdueBySchedule', 'PlannedPrin

### 2.b) Generate Statistics for Training Data

In [11]:
train_stats = tfdv.generate_statistics_from_dataframe(df_train, stats_options=options_train)

# get the number of features used to compute statistics
print(f"Number of features used: {len(train_stats.datasets[0].features)}")

# check the number of examples used
print(f"Number of examples used: {train_stats.datasets[0].num_examples}")

# check the column names of the first and last feature
print(f"First feature: {train_stats.datasets[0].features[0].path.step[0]}")
print(f"Last feature: {train_stats.datasets[0].features[-1].path.step[0]}")

Number of features used: 81
Number of examples used: 143388
First feature: NewCreditCustomer
Last feature: ActiveLateLastPaymentCategory


### 2.c) Visualize Training Statistics

In [12]:
tfdv.visualize_statistics(train_stats)

### 2.d) Infer Training Schema

In [13]:
# Infer the data schema by using the training statistics previously generated
dataset_schema = tfdv.infer_schema(train_stats)

# Display the data schema
tfdv.display_schema(dataset_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'NewCreditCustomer',INT,required,,-
'ApplicationSignedHour',INT,required,,-
'ApplicationSignedWeekday',INT,required,,-
'VerificationType',FLOAT,optional,single,-
'LanguageCode',INT,required,,-
...,...,...,...,...
'NrOfScheduledPayments',FLOAT,optional,single,-
'ReScheduledOn',BYTES,optional,single,-
'PrincipalDebtServicingCost',FLOAT,optional,single,-


/home/tensorflowml/appdev/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Country',"'EE', 'ES', 'FI', 'SK'"
'EmploymentDurationCurrentEmployer',"'MoreThan5Years', 'Other', 'Retiree', 'TrialPeriod', 'UpTo1Year', 'UpTo2Years', 'UpTo3Years', 'UpTo4Years', 'UpTo5Years'"
'Rating',"'A', 'AA', 'B', 'C', 'D', 'E', 'F', 'HR'"
'target',"'Current', 'Late', 'Repaid'"
'ActiveLateCategory',"'1-7', '121-150', '151-180', '16-30', '180+', '31-60', '61-90', '8-15', '91-120'"
'WorseLateCategory',"'1-7', '121-150', '151-180', '16-30', '180+', '31-60', '61-90', '8-15', '91-120'"
'CreditScoreEsMicroL',"'M', 'M1', 'M10', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9'"
'ActiveLateLastPaymentCategory',"'1-7', '121-150', '151-180', '16-30', '180+', '31-60', '61-90', '8-15', '91-120'"


## Task 3 - Check Anomalies in Validation Dataset

### 3.a) Generate Statistics for Validation Data

In [14]:
val_stats = tfdv.generate_statistics_from_dataframe(df_val, stats_options=options_train)

# get the number of features used to compute statistics
print(f"Number of features used: {len(val_stats.datasets[0].features)}")

# check the number of examples used
print(f"Number of examples used: {val_stats.datasets[0].num_examples}")

# check the column names of the first and last feature
print(f"First feature: {val_stats.datasets[0].features[0].path.step[0]}")
print(f"Last feature: {val_stats.datasets[0].features[-1].path.step[0]}")

Number of features used: 81
Number of examples used: 17923
First feature: NewCreditCustomer
Last feature: ActiveLateLastPaymentCategory


### 3.b) Compare Validation with Training Statistics

In [15]:
tfdv.visualize_statistics(lhs_statistics=val_stats, rhs_statistics=train_stats,
                          lhs_name='VAL_DATASET', rhs_name='TRAIN_DATASET')

### 3.c) Detect Anomalies

In [16]:
val_anomalies = tfdv.validate_statistics(statistics=val_stats, schema=dataset_schema)
tfdv.display_anomalies(val_anomalies)

/home/tensorflowml/appdev/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


### 3.d) Fix Validation Data Anomalies in Schema

In [17]:
# Not applicable in this iteration of the data validation exercise

## Task 4 - Check Anomalies in Test Dataset

### 4.a) Generate Statistics for Test Data

In [18]:
# Define a new statistics options by the tfdv.StatsOptions class for the serving data by passing the previously inferred schema
options_test = tfdv.StatsOptions(schema=dataset_schema, infer_type_from_schema=True, feature_allowlist=features_to_keep)

In [19]:
# Generate serving dataset statistics
test_stats = tfdv.generate_statistics_from_dataframe(df_test, stats_options=options_test)

# get the number of features used to compute statistics
print(f"Number of features used: {len(test_stats.datasets[0].features)}")

# check the number of examples used
print(f"Number of examples used: {test_stats.datasets[0].num_examples}")

# check the column names of the first and last feature
print(f"First feature: {test_stats.datasets[0].features[0].path.step[0]}")
print(f"Last feature: {test_stats.datasets[0].features[-1].path.step[0]}")

Number of features used: 80
Number of examples used: 17924
First feature: NewCreditCustomer
Last feature: ActiveLateLastPaymentCategory


### 4.b) Compare Test with Training Statistics

In [20]:
test_anomalies = tfdv.validate_statistics(statistics=test_stats, schema=dataset_schema)
tfdv.display_anomalies(test_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'target',Column dropped,Column is completely missing


### 4.c) Fix Test Data Anomalies in Schema

In [21]:
# Not applicable in this iteration of the data validation exercise

### 4.d) Organize Schema based on Environment

In [22]:
# All features are by default in both TRAINING and SERVING environments.
dataset_schema.default_environment.append('TRAINING')
dataset_schema.default_environment.append('TESTING')

In [23]:
# Specify that 'target' feature is not in TEST environment.
tfdv.get_feature(dataset_schema, 'target').not_in_environment.append('TESTING')

# Re-calculate and re-display anomalies with the new environment parameters
test_anomalies = tfdv.validate_statistics(statistics=test_stats, schema=dataset_schema, environment='TESTING')
tfdv.display_anomalies(test_anomalies)

/home/tensorflowml/appdev/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## Task 5 - Check for Data Drift and Skew

In [24]:
# Not applicable in this iteration of the data validation exercise

## Task 6 - Display Stats for Data Slices

In [25]:
def split_datasets(dataset_list):
    '''
    split datasets.

            Parameters:
                    dataset_list: List of datasets to split

            Returns:
                    datasets: sliced data
    '''
    datasets = []
    for dataset in dataset_list.datasets:
        proto_list = DatasetFeatureStatisticsList()
        proto_list.datasets.extend([dataset])
        datasets.append(proto_list)
    return datasets

In [26]:
def display_stats_at_index(index, datasets):
    '''
    display statistics at the specified data index

            Parameters:
                    index : index to show the anomalies
                    datasets: split data

            Returns:
                    display of generated sliced data statistics at the specified index
    '''
    if index < len(datasets):
        print(datasets[index].datasets[0].name)
        tfdv.visualize_statistics(datasets[index])

In [27]:
def sliced_stats_for_slice_fn(slice_fn, approved_cols, dataframe, schema):
    '''
    generate statistics for the sliced data.

            Parameters:
                    slice_fn : slicing definition
                    approved_cols: list of features to pass to the statistics options
                    dataframe: pandas dataframe to slice
                    schema: the schema

            Returns:
                    slice_info_datasets: statistics for the sliced dataset
    '''
    # Set the StatsOptions
    slice_stats_options = tfdv.StatsOptions(schema=schema,
                                            slice_functions=[slice_fn],
                                            infer_type_from_schema=True,
                                            feature_allowlist=approved_cols)
    
    # Convert Dataframe to CSV since `slice_functions` works only with `tfdv.generate_statistics_from_csv`
    CSV_PATH = 'slice_sample.csv'
    dataframe.to_csv(CSV_PATH)
    
    # Calculate statistics for the sliced dataset
    sliced_stats = tfdv.generate_statistics_from_csv(CSV_PATH, stats_options=slice_stats_options)
    
    # Split the dataset using the previously defined split_datasets function
    slice_info_datasets = split_datasets(sliced_stats)
    
    return slice_info_datasets

In [28]:
# # Generate slice function for one of the features
# slice_fn = slicing_util.get_feature_value_slicer(features={'NewCreditCustomer': None})

# # Generate stats for the sliced dataset
# slice_datasets = sliced_stats_for_slice_fn(slice_fn, features_to_keep, dataframe=df_train, schema=dataset_schema)

# # Print name of slices for reference
# print(f'Statistics generated for:\n')
# print('\n'.join([sliced.datasets[0].name for sliced in slice_datasets]))

# # Display at index 1 as an example
# display_stats_at_index(1, slice_datasets)

## Task 7 - Finalize the Schema

In [29]:
# Create output directory
OUTPUT_DIR = "output"
file_io.recursive_create_dir(OUTPUT_DIR)

# Use TensorFlow text output format pbtxt to store the schema
schema_file = os.path.join(OUTPUT_DIR, 'schema.pbtxt')

# write_schema_text function expect the defined schema and output path as parameters
tfdv.write_schema_text(dataset_schema, schema_file) 

In [30]:
print ('Total time for the script:',(datetime.now() - start_time_script))

Total time for the script: 0:00:42.161264
